In [1]:
import os
import tqdm
import time
import random
import itertools
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch # batch_size channel width height
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid

## GPU 설정

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## SEED

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

## Custom Dataset

In [4]:
class ImageDataset(torch.utils.data.Dataset):
    
    def __init__(self, A_dir, B_dir, img_size=256):
        self.A_dir = photo_dir
        self.B_dir = gogh_dir
        self.A_imgs = [filename for filename in os.listdir(A_dir) if os.path.splitext(filename)[-1] in ('.jpg', '.png') ]
        self.B_imgs = [filename for filename in os.listdir(B_dir) if os.path.splitext(filename)[-1] in ('.jpg', '.png') ]
        self.transform = [ torchvision.transforms.Resize(int(img_size*1.15), Image.BICUBIC), # 이미지 크기를 조금 키우기
                           torchvision.transforms.RandomCrop(img_size), 
                           torchvision.transforms.RandomHorizontalFlip(),
                           torchvision.transforms.ToTensor(),  #  [0 - 255] --> [0 - 1.0]
                           torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
        self.transform = torchvision.transforms.Compose(self.transform)
        
    def __getitem__(self, index):
        A_img = self.transform(Image.open(os.path.join(self.A_dir, 
                                                       self.A_imgs[index % len(self.A_imgs)])).convert('RGB'))
        B_img = self.transform(Image.open(os.path.join(self.B_dir, 
                                                       self.B_imgs[random.randint(0, len(self.B_imgs) - 1)])).convert('RGB')) # 랜덤 샘플링
        return A_img, B_img
    
    def __len__(self):
        return max(len(self.A_dir), len(self.B_dir))

## DataLoader

In [5]:
photo_dir = '../data/scenary/'
gogh_dir = '../data/gogh/'
train_dataset = ImageDataset(photo_dir, gogh_dir)
valid_dataset = ImageDataset(photo_dir, gogh_dir)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True)

In [6]:
train_loader

## Residual Block

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()

        # 채널(channel) 크기는 그대로 유지
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # Pads the input tensor using the reflection of the input boundary
            nn.Conv2d(in_channels, in_channels, kernel_size=3),
            nn.InstanceNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, kernel_size=3),
            nn.InstanceNorm2d(in_channels),
        )

    def forward(self, x):
        return x + self.block(x)

## Generator

In [8]:
class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()

        channels = input_shape[0] # 입력 이미지의 채널 수: 3

        # 초기 Convolution layer
        out_channels = 64
        model = [nn.ReflectionPad2d(channels)]
        model.append(nn.Conv2d(channels, out_channels, kernel_size=7))
        model.append(nn.InstanceNorm2d(out_channels))
        model.append(nn.ReLU(inplace=True))
        in_channels = out_channels

        # Downsampling
        for _ in range(2):
            out_channels *= 2
            model.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)) # 너비와 높이가 2배씩 감소
            model.append(nn.InstanceNorm2d(out_channels))
            model.append(nn.ReLU(inplace=True))
            in_channels = out_channels
        # 출력: [256 X (4배 감소한 높이) X (4배 감소한 너비)]

        # 인코더와 디코더의 중간에서 Residual Blocks 사용 (차원 유지)
        for _ in range(num_residual_blocks):
            model.append(ResidualBlock(out_channels))

        # Upsampling
        for _ in range(2):
            out_channels //= 2
            model.append(nn.Upsample(scale_factor=2)) # 너비와 높이가 2배씩 증가
            model.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)) # 너비와 높이는 그대로
            model.append(nn.InstanceNorm2d(out_channels))
            model.append(nn.ReLU(inplace=True))
            in_channels = out_channels
        # 출력: [256 X (4배 증가한 높이) X (4배 증가한 너비)]

        # 출력 Convolution Block layer
        model.append(nn.ReflectionPad2d(channels))
        model.append(nn.Conv2d(out_channels, channels, kernel_size=7))
        model.append(nn.Tanh())

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

## Discriminator

In [9]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape 

        # Convolution Block
        def discriminator_block(in_channels, out_channels, normalize=True):
            layers = [nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)] # 너비와 높이가 2배씩 감소
            if normalize:
                layers.append(nn.InstanceNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False), # 출력: [64 X 128 X 128]
            *discriminator_block(64, 128), # 출력: [128 X 64 X 64]
            *discriminator_block(128, 256), # 출력: [256 X 32 X 32]
            *discriminator_block(256, 512), # 출력: [512 X 16 X 16]
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 3, kernel_size=4, padding=1) # 출력: [3 X 16 X 16]
        )
        # 최종 출력: [3 X (16배 감소한 높이) X (16배 감소한 너비)]

    def forward(self, img):
        return self.model(img)

## ReplayBuffer

Replay Buffer GAN 트레이닝을 진행하며 똑같은 샘플별로 성능을 살펴보면 트레이닝을 돌릴때마다 성능이 천차만별이다. 이 불안정성을 해결하기 위해 주기적으로 Generator가 만들어 놓은 사진을 다시 discriminator에게 보여줌, 이 부분은 Discriminator에게만 적용함

In [10]:
class ReplayBuffer:
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []

    # 새로운 이미지를 삽입하고, 이전에 삽입되었던 이미지를 반환하는 함수
    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            # 아직 버퍼가 가득 차지 않았다면, 현재 삽입된 데이터를 반환
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            # 버퍼가 가득 찼다면, 이전에 삽입되었던 이미지를 랜덤하게 반환
            else:
                if random.uniform(0, 1) > 0.5: # 확률은 50%
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element # 버퍼에 들어 있는 이미지 교체
                else:
                    to_return.append(element)
        return torch.cat(to_return)

## Learning Rate 조정
시간이 지남에 따라 학습률(learning rate) 조정(감소)

In [11]:
class LambdaLR:
    def __init__(self, n_epochs, decay_start_epoch):
        self.n_epochs = n_epochs # 전체 epoch
        self.decay_start_epoch = decay_start_epoch # 학습률 감소가 시작되는 epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)

## 가중치 초기화

In [12]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

## Generator와 Discriminator 초기화

In [13]:
# 생성자(generator)와 판별자(discriminator) 초기화
G_AB = GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9)
G_BA = GeneratorResNet(input_shape=(3, 256, 256), num_residual_blocks=9)
D_A = Discriminator(input_shape=(3, 256, 256))
D_B = Discriminator(input_shape=(3, 256, 256))

G_AB.cuda()
G_BA.cuda()
D_A.cuda()
D_B.cuda()

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (12): Conv2d(512, 3, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [14]:
# 가중치(weights) 초기화
G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (12): Conv2d(512, 3, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

## Loss Function

In [15]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

criterion_GAN.cuda()
criterion_cycle.cuda()
criterion_identity.cuda()

L1Loss()

## 파라미터 설정

In [16]:
n_epochs = 200 # 학습의 횟수(epoch) 설정
decay_epoch = 100
lr = 0.0002 # 학습률(learning rate) 설정

## optimizer

In [17]:
optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(0.5, 0.999))
optimizer_D_A  = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(0.5, 0.999))

## 학습률(learning rate) 업데이트 스케줄러 초기화

In [18]:
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, decay_epoch).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(n_epochs, decay_epoch).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(n_epochs, decay_epoch).step)

## Train

In [19]:
sample_interval = 50 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 설정

lambda_cycle = 10 # Cycle 손실 가중치(weight) 파라미터
lambda_identity = 5 # Identity 손실 가중치(weight) 파라미터

# 이전에 생성된 이미지 데이터를 포함하고 있는 버퍼(buffer) 객체
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

start_time = time.time()

for epoch in range(n_epochs):
    for i, batch in enumerate(train_loader):
        # 모델의 입력(input) 데이터 불러오기
        real_A, real_B = batch
        real_A = real_A.cuda()
        real_B = real_B.cuda()

        # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 정답 레이블 생성 (너바와 높이를 16씩 나눈 크기)
        real = torch.cuda.FloatTensor(real_A.size(0), 1, 16, 16).fill_(1.0) # 진짜(real): 1
        fake = torch.cuda.FloatTensor(real_A.size(0), 1, 16, 16).fill_(0.0) # 가짜(fake): 0

        """ 생성자(generator)를 학습합니다. """
        G_AB.train()
        G_BA.train()

        optimizer_G.zero_grad()

        # Identity 손실(loss) 값 계산
        loss_identity_A = criterion_identity(G_BA(real_A), real_A)
        loss_identity_B = criterion_identity(G_AB(real_B), real_B)
        loss_identity = (loss_identity_A + loss_identity_B) / 2

        # GAN 손실(loss) 값 계산
        fake_B = G_AB(real_A)
        fake_A = G_BA(real_B)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), real)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), real)
        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # Cycle 손실(loss) 값 계산
        recover_A = G_BA(fake_B)
        recover_B = G_AB(fake_A)
        loss_cycle_A = criterion_cycle(recover_A, real_A)
        loss_cycle_B = criterion_cycle(recover_B, real_B)
        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # 최종적인 손실(loss)
        loss_G = loss_GAN + lambda_cycle * loss_cycle + lambda_identity * loss_identity

        # 생성자(generator) 업데이트
        loss_G.backward()
        optimizer_G.step()

        """ 판별자(discriminator) A를 학습합니다. """
        optimizer_D_A.zero_grad()

        # Real 손실(loss): 원본 이미지를 원본으로 판별하도록
        loss_real = criterion_GAN(D_A(real_A), real)

        # Fake 손실(loss): 가짜 이미지를 가짜로 판별하도록
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)

        # 최종적인 손실(loss)
        loss_D_A = (loss_real + loss_fake) / 2

        # 판별자(discriminator) 업데이트
        loss_D_A.backward()
        optimizer_D_A.step()

        """ 판별자(discriminator) B를 학습합니다. """
        optimizer_D_B.zero_grad()

        # Real 손실(loss): 원본 이미지를 원본으로 판별하도록
        loss_real = criterion_GAN(D_B(real_B), real)

        # Fake 손실(loss): 가짜 이미지를 가짜로 판별하도록
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)

        # 최종적인 손실(loss)
        loss_D_B = (loss_real + loss_fake) / 2

        # 판별자(discriminator) 업데이트
        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        done = epoch * len(train_loader) + i
        if done % sample_interval == 0:
            G_AB.eval()
            G_BA.eval()
            imgs = next(iter(val_loader)) # 5개의 이미지를 추출해 생성
            real_A, real_B = batch
            real_A = real_A.cuda()
            real_B = real_B.cuda()
            fake_B = G_AB(real_A)
            fake_A = G_BA(real_B)
            
            # X축을 따라 각각의 그리디 이미지 생성
            real_A = make_grid(real_A, nrow=4, normalize=True)
            real_B = make_grid(real_B, nrow=4, normalize=True)
            fake_A = make_grid(fake_A, nrow=4, normalize=True)
            fake_B = make_grid(fake_B, nrow=4, normalize=True)
            
            # 각각의 격자 이미지를 높이(height)를 기준으로 연결하기 
            image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
            save_image(image_grid, f"./generate_gogh/{done}.png", normalize=False)
            print(f"[Done {i}/{len(train_loader)}] [Elapsed time: {time.time() - start_time:.2f}s]")

    # 학습률(learning rate)
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    # 하나의 epoch이 끝날 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {loss_D.item():.6f}] [G identity loss: {loss_identity.item():.6f}, adv loss: {loss_GAN.item()}, cycle loss: {loss_cycle.item()}] [Elapsed time: {time.time() - start_time:.2f}s]")

    # 하나의 epoch이 끝날 때마다 모델 파라미터 저장
    torch.save(G_AB.state_dict(), "G_AB_gogh.pt")
    torch.save(G_BA.state_dict(), "G_BA_gogh.pt")
    torch.save(D_A.state_dict(), "D_A_gogh.pt")
    torch.save(D_B.state_dict(), "D_B_gogh.pt")
    print("Model saved!")

/home/danbibibi/.conda/envs/dev/lib/python3.6/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([4, 1, 16, 16])) that is different to the input size (torch.Size([4, 3, 16, 16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[Done 0/4] [Elapsed time: 2.51s]
[Epoch 0/200] [D loss: 1.434710] [G identity loss: 0.457904, adv loss: 1.5701693296432495, cycle loss: 0.5568963885307312] [Elapsed time: 7.54s]
Model saved!
[Epoch 1/200] [D loss: 0.920286] [G identity loss: 0.431080, adv loss: 0.8508347272872925, cycle loss: 0.49207034707069397] [Elapsed time: 13.27s]
Model saved!
[Epoch 2/200] [D loss: 0.810025] [G identity loss: 0.385519, adv loss: 0.7768833637237549, cycle loss: 0.41738617420196533] [Elapsed time: 19.33s]
Model saved!
[Epoch 3/200] [D loss: 0.626251] [G identity loss: 0.382108, adv loss: 0.5760961174964905, cycle loss: 0.4016782343387604] [Elapsed time: 25.14s]
Model saved!
[Epoch 4/200] [D loss: 0.522142] [G identity loss: 0.295782, adv loss: 0.4969215989112854, cycle loss: 0.32707515358924866] [Elapsed time: 31.30s]
Model saved!
[Epoch 5/200] [D loss: 0.431621] [G identity loss: 0.287795, adv loss: 0.3994126319885254, cycle loss: 0.3128988742828369] [Elapsed time: 37.17s]
Model saved!
[Epoch 6/20

[Epoch 50/200] [D loss: 0.238599] [G identity loss: 0.223189, adv loss: 0.3298575282096863, cycle loss: 0.24992907047271729] [Elapsed time: 308.98s]
Model saved!
[Epoch 51/200] [D loss: 0.244643] [G identity loss: 0.220026, adv loss: 0.36597740650177, cycle loss: 0.2584075331687927] [Elapsed time: 314.91s]
Model saved!
[Epoch 52/200] [D loss: 0.242432] [G identity loss: 0.198208, adv loss: 0.37587112188339233, cycle loss: 0.22058835625648499] [Elapsed time: 320.70s]
Model saved!
[Epoch 53/200] [D loss: 0.222375] [G identity loss: 0.197530, adv loss: 0.2907327711582184, cycle loss: 0.21548958122730255] [Elapsed time: 326.70s]
Model saved!
[Epoch 54/200] [D loss: 0.249572] [G identity loss: 0.201292, adv loss: 0.35156676173210144, cycle loss: 0.22842523455619812] [Elapsed time: 332.66s]
Model saved!
[Epoch 55/200] [D loss: 0.270036] [G identity loss: 0.209923, adv loss: 0.3852035403251648, cycle loss: 0.23129390180110931] [Elapsed time: 338.80s]
Model saved!
[Epoch 56/200] [D loss: 0.239

[Done 0/4] [Elapsed time: 604.15s]
[Epoch 100/200] [D loss: 0.215894] [G identity loss: 0.179788, adv loss: 0.4304780960083008, cycle loss: 0.20141692459583282] [Elapsed time: 608.55s]
Model saved!
[Epoch 101/200] [D loss: 0.192917] [G identity loss: 0.163414, adv loss: 0.34288012981414795, cycle loss: 0.17777732014656067] [Elapsed time: 614.13s]
Model saved!
[Epoch 102/200] [D loss: 0.209128] [G identity loss: 0.176305, adv loss: 0.43422731757164, cycle loss: 0.19656610488891602] [Elapsed time: 620.32s]
Model saved!
[Epoch 103/200] [D loss: 0.231178] [G identity loss: 0.175307, adv loss: 0.42305007576942444, cycle loss: 0.20929986238479614] [Elapsed time: 626.50s]
Model saved!
[Epoch 104/200] [D loss: 0.226062] [G identity loss: 0.175953, adv loss: 0.4105236232280731, cycle loss: 0.19173309206962585] [Elapsed time: 632.43s]
Model saved!
[Epoch 105/200] [D loss: 0.238193] [G identity loss: 0.175539, adv loss: 0.38359808921813965, cycle loss: 0.1990567296743393] [Elapsed time: 638.54s]


[Done 0/4] [Elapsed time: 903.55s]
[Epoch 150/200] [D loss: 0.200284] [G identity loss: 0.170685, adv loss: 0.35765784978866577, cycle loss: 0.1718934327363968] [Elapsed time: 907.61s]
Model saved!
[Epoch 151/200] [D loss: 0.166320] [G identity loss: 0.132304, adv loss: 0.5516176223754883, cycle loss: 0.14101043343544006] [Elapsed time: 913.36s]
Model saved!
[Epoch 152/200] [D loss: 0.192002] [G identity loss: 0.143869, adv loss: 0.41252097487449646, cycle loss: 0.14714446663856506] [Elapsed time: 919.12s]
Model saved!
[Epoch 153/200] [D loss: 0.165780] [G identity loss: 0.165972, adv loss: 0.4353264272212982, cycle loss: 0.1699172556400299] [Elapsed time: 924.91s]
Model saved!
[Epoch 154/200] [D loss: 0.177007] [G identity loss: 0.136117, adv loss: 0.4658746123313904, cycle loss: 0.14732509851455688] [Elapsed time: 930.49s]
Model saved!
[Epoch 155/200] [D loss: 0.162348] [G identity loss: 0.161125, adv loss: 0.3987753093242645, cycle loss: 0.1743139922618866] [Elapsed time: 936.26s]
M